In [ ]:
!pip install xgboost lightgbm pandas numpy scikit-learn matplotlib

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import lightgbm as lgb

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error

import warnings

warnings.filterwarnings("ignore")

In [ ]:
filepath = 'cityD-dataset.csv'

df=pd.read_csv(filepath)


df.head()

NameError: name 'pd' is not defined

In [ ]:
df.shape

(8418135, 5)

In [ ]:
df.tail()

,uid,d,t,x,y
8418130,5999,74,37,97,121
8418131,5999,74,38,110,120
8418132,5999,74,39,111,111
8418133,5999,74,46,110,110
8418134,5999,74,47,110,110


In [ ]:
df.shape

(8418135, 5)

In [ ]:
import numpy as np
df["week"]=df["d"]//7
df["day_of_week"]=df["d"]%7
weekend_list=[]
for i in range(0,75,7):
    weekend_list.extend([i,i+1])
print("weekend list:", weekend_list)
df["weekend"]=np.where(df["d"].isin(weekend_list),1,0)

df_group=df.groupby(["uid","x","y"]).size().reset_index(name="count_coords_total_num")
df=df.merge(df_group)

df_group=df.groupby(["uid","d","x","y"]).size().reset_index(name="count_coords_by_day")
df=df.merge(df_group)

df_group=df.groupby(["uid","t","x","y"]).size().reset_index(name="count_coords_by_time")
df=df.merge(df_group)

df_group=df.groupby(["uid","weekend","x","y"]).size().reset_index(name="count_coords_by_weekend")
df=df.merge(df_group)

df_group=df.groupby(["uid","day_of_week","x","y"]).size().reset_index(name="count_coords_by_day_of_week")
df=df.merge(df_group)

df

weekend list: [0, 1, 7, 8, 14, 15, 21, 22, 28, 29, 35, 36, 42, 43, 49, 50, 56, 57, 63, 64, 70, 71]


,uid,d,t,x,y,week,day_of_week,weekend,count_coords_total_num,count_coords_by_day,count_coords_by_time,count_coords_by_weekend,count_coords_by_day_of_week
0,0,0,13,133,103,0,0,1,6,1,1,2,1
1,0,0,14,143,98,0,0,1,117,4,2,37,21
2,0,0,16,162,99,0,0,1,5,1,2,3,2
3,0,0,17,149,97,0,0,1,4,1,1,2,2
4,0,0,18,144,98,0,0,1,92,4,5,32,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8418130,5999,74,37,97,121,10,4,0,2,1,1,2,1
8418131,5999,74,38,110,120,10,4,0,3,1,1,2,1
8418132,5999,74,39,111,111,10,4,0,57,1,4,40,3
8418133,5999,74,46,110,110,10,4,0,17,2,3,12,4


In [ ]:
df_train=df[df["d"]<60]
df_test=df[df["d"]>=60]

In [ ]:
df_train.shape

(6696506, 13)

In [ ]:
df_test.shape

(1721629, 13)

In [ ]:
df.columns

Index(['uid', 'd', 't', 'x', 'y', 'week', 'day_of_week', 'weekend',
       'count_coords_total_num', 'count_coords_by_day', 'count_coords_by_time',
       'count_coords_by_weekend', 'count_coords_by_day_of_week'],
      dtype='object')

In [ ]:
features = ['uid', 'd', 't', 'week', 'day_of_week', 'weekend', 'count_coords_total_num', 'count_coords_by_day', 'count_coords_by_time', 'count_coords_by_weekend', 'count_coords_by_day_of_week']

X_train = df_train[features].to_numpy()
y1_train = df_train['x'].to_numpy()
y2_train = df_train['y'].to_numpy()

X_test = df_test[features].to_numpy()
y1_test = df_test['x'].to_numpy()
y2_test = df_test['y'].to_numpy()

In [ ]:
# 定义模型训练和预测函数
def train_and_predict(model, X_train, y_train, X_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return y_pred

# 评估函数
def evaluate(y_true, y_pred, label=""):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    print(f"{label} - MSE: {mse:.4f}, MAE: {mae:.4f}")
    return mse, mae

In [ ]:
# 结果dataframe
def create_prediction_df(y1_true, y1_pred, y2_true, y2_pred):
    return pd.DataFrame({
        'True_x': y1_true,
        'Predicted_x': y1_pred,
        'True_y': y2_true,
        'Predicted_y': y2_pred
    })

In [ ]:
# 随机森林模型
rf_model_x = RandomForestClassifier(
    n_estimators=300,
    max_depth=20,
    min_samples_leaf=5,
    random_state=0,
    n_jobs=-1
)

rf_model_y = RandomForestClassifier(
    n_estimators=300,
    max_depth=20,
    min_samples_leaf=5,
    random_state=0,
    n_jobs=-1
)

rf_y_pred1 = train_and_predict(rf_model_x, X_train, y1_train, X_test)
rf_y_pred2 = train_and_predict(rf_model_y, X_train, y2_train, X_test)

In [ ]:
# 随机森林结果DataFrame
rf_pred_df = create_prediction_df(y1_test, rf_y_pred1, y2_test, rf_y_pred2)

# 保存到本地
rf_pred_df.to_csv('rf_predictions.csv', index=False)
rf_pred_df

,True_x,Predicted_x,True_y,Predicted_y
0,129,129,102,99
1,143,143,98,98
2,143,129,102,100
3,141,129,106,93
4,125,129,129,93
...,...,...,...,...
1721624,97,97,121,108
1721625,110,93,120,117
1721626,111,165,111,111
1721627,110,101,110,104


In [ ]:
# 评估结果
rf_x_mse, rf_x_mae = evaluate(y1_test, rf_y_pred1, label="rf_x_performance")
rf_y_mse, rf_y_mae = evaluate(y2_test, rf_y_pred2, label="rf_y_performance")

rf_avg_mse = (rf_x_mse + rf_y_mse) / 2
rf_avg_mae = (rf_x_mae + rf_y_mae) / 2

rf_x_performance - MSE: 1538.9449, MAE: 23.4855
rf_y_performance - MSE: 1255.7946, MAE: 21.2191


In [ ]:
# LightGBM 模型
lgb_model_x = lgb.LGBMClassifier(
    n_estimators=300,
    max_depth=20,
    num_leaves=31,
    learning_rate=0.05,
    random_state=0,
    n_jobs=-1
)
lgb_model_y = lgb.LGBMClassifier(
    n_estimators=300,
    max_depth=20,
    num_leaves=31,
    learning_rate=0.05,
    random_state=0,
    n_jobs=-1
)

lgb_y_pred1 = train_and_predict(lgb_model_x, X_train, y1_train, X_test)
lgb_y_pred2 = train_and_predict(lgb_model_y, X_train, y2_train, X_test)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.056660 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1258
[LightGBM] [Info] Number of data points in the train set: 6696506, number of used features: 11
[LightGBM] [Info] Start training from score -7.138996
[LightGBM] [Info] Start training from score -8.188227
[LightGBM] [Info] Start training from score -7.739471
[LightGBM] [Info] Start training from score -7.915705
[LightGBM] [Info] Start training from score -6.991914
[LightGBM] [Info] Start training from score -8.460799
[LightGBM] [Info] Start training from score -8.503328
[LightGBM] [Info] Start training from score -7.776157
[LightGBM] [Info] Start training from score -8.436399
[LightGBM] [Info] Start training from score -8.011834
[LightGBM] [Info] Start training from score -7.351657
[LightGBM] [Info] Start training from score -8.0939

In [ ]:
# LightGBM结果DataFrame
lgb_pred_df = create_prediction_df(y1_test, lgb_y_pred1, y2_test, lgb_y_pred2)
lgb_pred_df.to_csv('lgb_predictions.csv', index=False)
lgb_pred_df

,True_x,Predicted_x,True_y,Predicted_y
0,129,111,102,60
1,143,157,98,25
2,143,124,102,106
3,141,193,106,123
4,125,193,129,123
...,...,...,...,...
1721624,97,90,121,35
1721625,110,111,120,116
1721626,111,184,111,88
1721627,110,111,110,111


In [ ]:
lgb_x_mse, lgb_x_mae = evaluate(y1_test, lgb_y_pred1, label="lgb_x_performance")
lgb_y_mse, lgb_y_mae = evaluate(y2_test, lgb_y_pred2, label="lgb_x_performance")
lgb_avg_mse = (lgb_x_mse + lgb_y_mse) / 2
lgb_avg_mae = (lgb_x_mae + lgb_y_mae) / 2

lgb_x_performance - MSE: 4377.5125, MAE: 52.6040
lgb_x_performance - MSE: 3596.3328, MAE: 47.8932


In [ ]:
# AdaBoost 模型
ada_model_x = AdaBoostClassifier(
    n_estimators=200,
    learning_rate=0.5,
    random_state=0
)
ada_model_y = AdaBoostClassifier(
    n_estimators=200,
    learning_rate=0.5,
    random_state=0
)

ada_y_pred1 = train_and_predict(ada_model_x, X_train, y1_train, X_test)
ada_y_pred2 = train_and_predict(ada_model_y, X_train, y2_train, X_test)

In [ ]:
# AdaBoost结果DataFrame
ada_pred_df = create_prediction_df(y1_test, ada_y_pred1, y2_test, ada_y_pred2)
ada_pred_df.to_csv('ada_predictions.csv', index=False)
ada_pred_df

,True_x,Predicted_x,True_y,Predicted_y
0,129,117,102,92
1,143,126,98,92
2,143,117,102,92
3,141,117,106,92
4,125,117,129,92
...,...,...,...,...
1721624,97,2,121,118
1721625,110,2,120,118
1721626,111,101,111,118
1721627,110,101,110,118


In [ ]:
ada_x_mse, ada_x_mae = evaluate(y1_test, ada_y_pred1, label="ada_x_performance")
ada_y_mse, ada_y_mae = evaluate(y2_test, ada_y_pred2, label="ada_x_performance")
ada_avg_mse = (ada_x_mse + ada_y_mse) / 2
ada_avg_mae = (ada_x_mae + ada_y_mae) / 2

ada_x_performance - MSE: 3596.0208, MAE: 45.7301
ada_x_performance - MSE: 2329.5690, MAE: 37.0979


In [ ]:
# 将结果放入 DataFrame
results = {
    'Model': ['Random Forest', 'LightGBM', 'AdaBoost'],
    'X_MSE': [rf_x_mse, lgb_x_mse, ada_x_mse],
    'X_MAE': [rf_x_mae, lgb_x_mae, ada_x_mae],
    'Y_MSE': [rf_y_mse, lgb_y_mse, ada_y_mse],
    'Y_MAE': [rf_y_mae, lgb_y_mae, ada_y_mae],
    'Avg_MSE': [rf_avg_mse, lgb_avg_mse, ada_avg_mse],
    'Avg_MAE': [rf_avg_mae, lgb_avg_mae, ada_avg_mae]
}
# 转成DataFrame
results_df = pd.DataFrame(results)
# 打印表格
results_df



,Model,X_MSE,X_MAE,Y_MSE,Y_MAE,Avg_MSE,Avg_MAE
0,Random Forest,1538.944945,23.485490,1255.794585,21.219087,1397.369765,22.352288
1,LightGBM,4377.512452,52.604045,3596.332795,47.893220,3986.922623,50.248633
2,AdaBoost,3596.020777,45.730107,2329.569001,37.097859,2962.794889,41.413983


In [ ]:


results_df.to_csv('results_mse_mae.csv', index=False)

In [ ]:
# 各个模型保存

import pickle

with open('rf_model_x.pkl', 'wb') as f:
    pickle.dump(rf_model_x, f)

with open('rf_model_y.pkl', 'wb') as f:
    pickle.dump(rf_model_y, f)

with open('lgb_model_x.pkl', 'wb') as f:
    pickle.dump(lgb_model_x, f)

with open('lgb_model_y.pkl', 'wb') as f:
    pickle.dump(lgb_model_y, f)

with open('ada_model_x.pkl', 'wb') as f:
    pickle.dump(ada_model_x, f)

with open('ada_model_y.pkl', 'wb') as f:
    pickle.dump(ada_model_y, f)
